In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [120]:
def get_links(i):
    links=[]
    titles=[]
    r = requests.get('https://insideclimatenews.org/news/14052015/whats-behin?page={}'.format(i))
    soup = BeautifulSoup(r.text, 'html.parser')
    article_list = soup.find('div', {'class':'pane-content'})
    
    for a in article_list.findAll('a'):
        if a.parent.name=='h3':
            links.append('https://insideclimatenews.org'+a['href'])
            titles.append(a.text)
    return links, titles

def get_article(link):
    article=''
    author=''
    date=''
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'html.parser')
    article_body = soup.find('div', {'class':'pane-content'})
    if not article_body:
        return article, author, date
    #get author and date
    entry_meta = article_body.find('div', {'class':'entry-meta'})
    if not entry_meta:
        return article, author, date
    #try h4 or p
    p_author = entry_meta.find('p', {'class':'byline'})
    h4_author = entry_meta.find('h4', {'class':'byline'})
    if p_author:
        author = p_author.text.replace("By ","")
    elif h4_author:
        author = h4_author.text.replace("By ","")
    date=entry_meta.find('div',{'class':'posted-on'}).text.strip()
    subtitle=article_body.find('h2',{'class':'page-subtitle'})
    if subtitle:
        article+=subtitle.text
    [s.extract() for s in soup('header')]
    [s.extract() for s in soup('ul')]
    article_body = soup.find('div', {'class':'pane-content'})
    if article_body:
        widget = article_body.find('div', {'class':"widget"})
        if widget:
            widget.decompose()
        media = article_body.find('div', {'class':"media"})
        if media:
            media.decompose()
        tagged_with = article_body.find('div', {'class':"tagged-with"})
        if tagged_with:
            tagged_with.decompose()
        field_type_image = article_body.find('div', {'class':'field-type-image'})
        if field_type_image:
            field_type_image.decompose()
        for p in article_body.findAll('p'):
            article = article + '¶' + p.text.replace(u'\xa0', u' ')
        
    return article, author, date

In [115]:
article, author, date=get_article('https://insideclimatenews.org/news/23082019/south-portland-oil-tank-pollution-testing-results-voc-benzene-source')

In [118]:
date

'Aug 23, 2019'

In [121]:
inside_climate_news_articles=[]
inside_climate_news_dates=[]
inside_climate_news_authors=[]
inside_climate_news_titles=[]
for i in range(444):
    if i%50 == 0:
        print(i)
    links, titles =get_links(i)
    inside_climate_news_titles.extend(titles)
    for link in links:
        article, author, date= get_article(link)
        inside_climate_news_articles.append(article)
        inside_climate_news_authors.append(author)
        inside_climate_news_dates.append(date)
        

0
50
100
150
200
250
300
350
400


AttributeError: 'NoneType' object has no attribute 'findAll'

In [60]:
get_article("https://insideclimatenews.org/news/28082019/climate-change-river-flood-risk-europe-extreme-weather-magnitude-regions-study")

'\nBritish soldiers and mountain rescue teams evacuated people in York, UK, as the River Ouse flooded in 2015. Credit: Ian Forsyth/Getty Images\nGlobal warming is driving big changes in floods across Europe by fueling the atmosphere with more moisture and changing the path and speed of rain storms, new research shows.\nIn some areas, that means more rainfall and surging rivers that could overwhelm levees if communities don\'t plan for increasing flooding. Other regions have seen a decline in rain and snow, which sets up a different challenge: as flood risk there decreases, it could discourage investments in defensive measures, leaving communities vulnerable to less frequent but still damaging extreme storms.\nThe study shows "clear flood risk patterns across Europe that match the projected impacts of climate change," said Günther Blöschl, lead author of the study, published Wednesday in the journal Nature, and director of the Centre for Water Resource Systems at the Vienna University o

In [122]:
print(len(inside_climate_news_articles))
print(len(inside_climate_news_titles))
print(len(inside_climate_news_dates))
print(len(inside_climate_news_authors))

4426
4426
4426
4426


In [134]:
l=[i for i in range(len(inside_climate_news_titles)) if inside_climate_news_dates[i] == '']
l

[]

In [130]:
i=0
for entry in l:
    del inside_climate_news_articles[entry-i]
    del inside_climate_news_authors[entry-i]
    del inside_climate_news_titles[entry-i]
    del inside_climate_news_dates[entry-i]
    i+=1

In [143]:
df = pd.DataFrame(inside_climate_news_articles, columns=['content'])
df['title']=inside_climate_news_titles
df['author']=inside_climate_news_authors
df['date']=inside_climate_news_dates
df['denial?'] = [0]*len(df.index)
cols=['title','author','date','content','denial?']
df=df[cols]
df = df.drop_duplicates()
df['content'] = df['content'].str.strip()
df.to_csv('../../data/left/data_inside_climate_news_articles.csv', index=False)

In [144]:
df

,title,author,date,content,denial?
0,Trump's EPA Targets Methane Rules for Oil and ...,Marianne Lavelle,"Aug 29, 2019",The proposed rule change involves leaks of the...,0
1,Climate Change Is Shifting Europe's Flood Patt...,"Bob Berwyn, InsideClimate News","Aug 28, 2019",A new study maps out which regions have seen i...,0
2,BP's Selling Off Its Alaska Oil Assets. The Bu...,Sabrina Shankman,"Aug 28, 2019",BP’s stake in the Trans Alaska Pipeline and th...,0
3,What Science Says About That '12 Years' Climat...,"Bob Berwyn, InsideClimate News","Aug 27, 2019",It doesn't mean the world can wait until 2030 ...,0
4,City's Air Testing Reveals Troubling Benzene S...,Sabrina Shankman,"Aug 23, 2019",¶Episode 4: The Fumes in South Portland. The f...,0
5,Amazon Fires Spark Growing International Criti...,"ARTHUR BEESLEY & VICTOR MALLET, FINANCIAL TIMES","Aug 23, 2019",France calls the large number of fires in the ...,0
6,Western Coal Takes Another Hit as Appeals Cour...,Phil McKenna,"Aug 23, 2019",Western coal states want an export terminal on...,0
7,Sanders Unveils $16 Trillion Green New Deal Pl...,Nicholas Kusnetz,"Aug 22, 2019",It’s the most ambitious climate program of any...,0
8,Environmental Groups Sue to Block Trump's Enda...,Georgina Gustin,"Aug 22, 2019",The changes make it harder to take future risk...,0
9,We’re Investigating Heat Deaths and Illnesses ...,David Hasemyer,"Aug 22, 2019",¶InsideClimate News and NBC News spent the pas...,0
